In [149]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
import parmap
import numpy as np
import pandas as pd
import os
import shutil
import cv2
import scipy.io as sio
import importlib

import glob2

from numba import jit
from sklearn.svm import SVC # "Support vector classifier"
import matplotlib.patches as mpatches

colors = [
'blue','red','green','cyan',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']



Autosaving every 180 seconds


In [148]:
# NATURAL BEHAVIOUR

n_time_steps = 2500
bump_threshold = 0.002
bump_earliest = n_time_steps//2
awareness = 20
action_time = 100
n_decisions = 5

# internal state scaling
# cued scaling 
internal_evidence_scaling= 30
external_evidence_scaling= external_evidence_scaling

# bumps model values
bumps_scaling = 5
bumps_frequency = 10

clrs = ['blue','red','green','magenta','brown']

evidence = np.zeros((n,n_time_steps))

# starting points; cued vs. uncued
evidence_bias = 800
evidence[:,0] = np.random.rand(evidence.shape[0])*evidence_bias-(evidence_bias-awareness)
#evidence[:,0]=-evidence_bias/4

# population biases that limit evidence accumulation per population
population_scaling =np.random.rand(n_decisions)*2

# check if awareness bump has occuredd
bump_states = [False,False,False,False,False]
bump_times = [None,None,None,None,None]

# Simulate BUMPS
from scipy.signal import hilbert, chirp

duration = n_time_steps
fs = 1500.0
samples = int(fs*duration)
t = np.arange(samples) / fs
signal = chirp(t, 20.0, t[-1], 100.0)
signal *= (1.0 + 0.5 * np.sin(2.0*np.pi*3.0*t) )*20

# place where leaks are weaker
turning_point = np.random.randint(n_time_steps//2)
    
ctr = 1
times = []
fig=plt.figure()
ax=plt.subplot(111)
while True:

    choice_threshold = np.random.normal(0.5,0.15,1)
    
    for p in range(n_decisions):
        
        # inherit previous amount of evidence
        evidence[p,ctr]=evidence[p,ctr]+evidence[p,ctr-1]

        # decide of choice is evaluate at time step; not all choices evaluate at all time steps internall/externally
        choice_evaluation = np.random.rand()
        if choice_evaluation>0.45:
            plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5)
            continue
        
        # model instantanenous evidence gathering
        # moment by moment adustment of whether new evidence will be added at time step
        #choice_bias = np.random.normal(0.5,0.15,1)
        choice = np.random.rand()
        evidence_amount = np.random.rand()   #May wish to model negative evidence
        if choice>choice_threshold:
            
            # if subconsciouss, the evidence will be weaker more diffuse?
            if evidence[p,ctr]<awareness:
                evidence[p,ctr] += evidence_amount*internal_evidence_scaling*population_scaling[p]
            else:
                evidence[p,ctr] += evidence_amount*external_evidence_scaling*population_scaling[p]
      
        #model leak; 
        #after turning point there is a lower leak
        #if ctr<turning_point:
        if False:
            if evidence[p,ctr]<awareness:
                evidence[p,ctr]-=(np.random.rand()/2)*internal_evidence_scaling
            else:
                evidence[p,ctr]-=(np.random.rand()/2)*external_evidence_scaling
        else:
        
            if evidence[p,ctr]<awareness:
                evidence[p,ctr]-=(np.random.rand()/2.2)*internal_evidence_scaling*population_scaling[p]
            else:
                evidence[p,ctr]-=(np.random.rand()/2.)*external_evidence_scaling*population_scaling[p]
        
        
        # bumps model
        evidence[p,ctr] += evidence_amount*signal[ctr]*population_scaling[p]
        
        # awarenss discounting/aggregation
        if False:
            bump = np.random.rand()
            if bump<0.1 and bump_states[p]==False and evidence[p,ctr]>awareness:
                evidence[p,ctr]+= np.random.rand()*external_evidence_scaling 
                #bump_states[p] = True
                #bump_times[p] = ctr

                plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=10)

        plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5)
        #print (p, evidence[p,ctr])
    #if bump_time is not None:
    #    plt.plot([bump_time,bump_time],[0,50],'r--',c=clrs[p],linewidth=3)
    
    ctr+=1

    if ctr>n_time_steps-1:
        break
    
    if np.max(evidence)>action_time:
        for z in range(n_decisions):
            print (evidence[z,ctr-1], action_time)
            if evidence[z,ctr-1]>=action_time:
                plt.scatter(ctr-1, evidence[z,ctr-1],s=300, c=clrs[z])
        break

ctr-=1
for p in range(n_decisions):
    plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5, label="Action "+str(p+1))

#plt.plot([0,ctr*1.1],[awareness,awareness],'r--',c='black')
plt.plot([0,ctr*1.1],[action_time,action_time],'r--',linewidth=5,c='black')

# visualize underlying waves
#sin_f = np.sin(np.arange(ctr)/bumps_frequency)*bumps_scaling*50+np.min(evidence)*1.5
sin_f = signal*5-evidence_bias*1.2
plt.plot(sin_f,c='black', linewidth=4, label="SCP")


# plotting data
x_width = 1.05
#ax.axhspan(np.min(sin_f)*1.1, action_time, alpha=0.1, color='black')

#ax.axhspan(awareness,action_time, alpha=0.1, color='red')

#ax.axhspan(action_time,action_time*x_width*3.2, alpha=0.1, color='green')

# formats
plt.ylim(np.min(sin_f)*1.1,action_time*x_width*3.2)
plt.xlabel("Time",fontsize=30)
plt.ylabel("Evidence",fontsize=30)
ax.set_xticks([])
#ax.set_yticks([])
#plt.yticks(fontsize=30)
#plt.legend(fontsize=20)
plt.xlim(0,ctr*x_width)
plt.suptitle("Natural Action Behaviuor Selection",fontsize=20)
plt.show()


-203.86057298560303 100
121.99610361487606 100
5.778692724117811 100
-36.93949830605486 100
-227.48994421063063 100


In [ ]:

ctr-=1
for p in range(n_decisions):
    plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5, label="Action "+str(p+1))

plt.plot([0,ctr*1.1],[awareness,awareness],'r--',c='black')
plt.plot([0,ctr*1.1],[action_time,action_time],'r--',c='black')

# visualize underlying waves
sin_f = np.sin(np.arange(ctr)/bumps_frequency)*bumps_scaling*50+np.min(evidence)*1.5
plt.plot(sin_f,c='black', linewidth=4)


# plotting data
x_width = 1.05
ax.axhspan(np.min(sin_f)*1.1, awareness, alpha=0.1, color='black')
ax.axhspan(awareness,action_time, alpha=0.1, color='red')
ax.axhspan(action_time,action_time*x_width*1.2, alpha=0.1, color='green')

# formats
plt.ylim(np.min(sin_f)*1.1,action_time*x_width*1.2)
plt.xlabel("Time",fontsize=30)
plt.ylabel("Evidence",fontsize=30)
ax.set_xticks([])
#ax.set_yticks([])
#plt.yticks(fontsize=30)
plt.legend(fontsize=30)
plt.xlim(0,ctr*x_width)
plt.suptitle("Natural Action Behaviuor Selection",fontsize=20)
plt.show()

In [19]:
# STIMULUS AND LAB BEHAVIOUR

n_time_steps = 3000
bump_threshold = 0.002
bump_earliest = n_time_steps//2
awareness_threshold = 100
action_threshold = 200
n_decisions = 10
n_trials = 25


# population amplitude; a weigth that repersents how much each population adds to the EEG signal
population_weights = np.ones(n_decisions,'float32')
#population_weights = np.random.rand(n_decisions)*5+1


# internal state scaling
evidence_scaling = np.zeros(n_decisions,'float32')
for k in range(n_decisions):
    #evidence_scaling[k]=10 
    evidence_scaling[k]=k*10/n_decisions+2


# bumps model values
bumps_scaling = 1
bumps_frequency = 80


stim_amount = 500

clrs = ['blue','red','green','magenta','brown']

evidence = np.zeros((n_trials,n_decisions,n_time_steps),'float32')+np.nan

# starting points; cued vs. uncued
evidence_bias = 0
#evidence[:,0] = np.random.rand(5)*evidence_bias-(evidence_bias-bump_awareness)
evidence[:,:,0]=-evidence_bias/4




# check if awareness bump has occuredd
bump_states = [False,False,False,False,False]
bump_times = [None,None,None,None,None]

# place where leaks are weaker
turning_point = np.random.randint(n_time_steps//2)
    
   
urgency = np.arange(n_time_steps)**2/1000
#urgency=np.zeros(n_time_steps)

# keep track which trials resulted in decisions
decision_reached = np.zeros(n_trials,'int32')
decision_type = np.zeros(n_trials,'int32')

# keep track of randomized stimulus noise
stim_noise = np.zeros((n_trials,n_time_steps),'float32')

# use single trial and add noise
stereotyped_trial = np.zeros((n_decisions,n_time_steps),'float32')
use_stereotyped = True

    
times = []
#traces = np.zeros((n_decisions, n_time_steps),'float32')+np.nan
for t in range(n_trials):
    ctr = 1
    while True:

        choice_threshold = np.random.normal(0.5,0.15,1)

        # add stimulus as white noise
        stim_noise[t,ctr] = np.random.normal(0,0.5)*stim_amount-stim_amount/2
        for p in range(n_decisions):
            
            # inherit previous amount of evidence
            evidence[t,p,ctr]=evidence[t,p,ctr-1]

            #evidence[t,p,ctr]+=stim_noise
            
            # add single neuron noise
            evidence[t,p,ctr]+=np.random.rand()*20-10
            
            
            # decide of choice is evaluate at time step; not all choices evaluate at all time steps internall/externally
            choice_evaluation = np.random.rand()
            #if choice_evaluation>0.45:
            if choice_evaluation>0.25:
                #plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5)
                continue

            # model instantanenous evidence gathering
            # moment by moment adustment of whether new evidence will be added at time step
            #choice_bias = np.random.normal(0.5,0.15,1)
            choice = np.random.rand()
            evidence_amount = np.random.rand()   #May wish to model negative evidence
            if choice>choice_threshold:

                # if subconsciouss, the evidence will be weaker more diffuse?
                #if evidence[p,ctr]<awareness:
                evidence[t,p,ctr] += evidence_amount*evidence_scaling[p]
                #else:
                #    evidence[p,ctr] += evidence_amount*external_evidence_scaling

            #model leak; 
            #after turning point there is a lower leak
            #if ctr<turning_point:
    #         if False:
    #             if evidence[p,ctr]<awareness:
    #                 evidence[p,ctr]-=(np.random.rand()/2)*internal_evidence_scaling
    #             else:
    #                 evidence[p,ctr]-=(np.random.rand()/2)*external_evidence_scaling
    #         else:
            # leak
            evidence[t,p,ctr]-=(np.random.rand()/2.1)*evidence_scaling[p]
            #else:
            #    evidence[p,ctr]-=(np.random.rand()/2.)*external_evidence_scaling

            
            if use_stereotyped==True and t>0:
                evidence[t,p,ctr] = stereotyped_trial[p,ctr]
                
            
            # bumps model
            if False:
                evidence[t,p,ctr] += np.sin(ctr/bumps_frequency)*bumps_scaling

            # awarenss discounting/aggregation
#             if False:
#                 bump = np.random.rand()
#                 if bump<0.1 and bump_states[p]==False and evidence[p,ctr]>awareness:
#                     evidence[p,ctr]+= np.random.rand()*external_evidence_scaling 
#                     #bump_states[p] = True
#                     #bump_times[p] = ctr

#                     plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=10)
            
            #evidence[t,p,ctr]+=urgency[ctr]

        
        # IS THIS CORRECT IN PRINCIPLE? 
        # check to see if any of the values are past threhsold
        
        if (np.nanmax(evidence[t,:,ctr])+
            #stim_noise[t,ctr]+
            urgency[ctr])>action_threshold:
            decision_reached[t]=1
            
            temp = np.argmax(evidence[t,:,ctr])
            decision_type[t]=temp
            print("Decision: ", temp)
            break

        ctr+=1

        if ctr>n_time_steps-1:
            break
    
    if t==0:
        stereotyped_trial=evidence[t]


#ctr-=1

Decision:  4
Decision:  4
Decision:  4
Decision:  0


<ipython-input-19-d5dc3f9ce752>:146: RuntimeWarning: All-NaN slice encountered
  if (np.nanmax(evidence[t,:,ctr])+


Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  0
Decision:  4
Decision:  4
Decision:  4
Decision:  4
Decision:  2


In [52]:

    
# **************************************
# plot EEG SIGNAL
# **************************************
if True:
    fig=plt.figure()
    ax2=plt.subplot(1,1,1)
    sums = np.zeros(n_time_steps, 'float32')+np.nan
    eeg_arrays = np.zeros((n_trials,n_time_steps),'float32')+np.nan

    # 
    idx2 = np.where(decision_reached)[0]
    
    eeg_array = []
    # PLOT EEG
    eeg_offset1 = 1200
    #plot eeg not single neuron data must adda
    for t in idx2:

        # find when decisino was made:
        idx = np.where(np.isnan(evidence[t,0])==False)[0]
        #print (idx[-1])
        eeg = (np.nansum((evidence[t,:,:idx[-1]].T*population_weights).T,0)
                        +urgency[:idx[-1]]
                        +stim_noise[t,:idx[-1]])
        
        #temp=np.nansum(evidence[t,:,:idx[-1]],0)
        x = np.arange(n_time_steps-eeg.shape[0],n_time_steps,1)
        plt.plot(x,eeg+eeg_offset1,alpha=0.1,c='green')
        #np.nansum(evidence[0,:ctr],0)+urgency,c='black',linewidth=5)
        eeg_arrays[t,x]=eeg

    total = np.nanmean(eeg_arrays,0)+eeg_offset1

    #print (total)
    idx = np.where(np.isnan(total)==False)[0]
    standard_deviation = np.nanstd(eeg_arrays,0)[idx[0]:]
    x=np.arange(idx[0],n_time_steps,1)

    plt.plot(x,total[idx[0]:],linewidth=5,c='green', label="EEG ")
    plt.fill_between(x, total[idx[0]:] - standard_deviation, total[idx[0]:] + standard_deviation,
                    color='green',alpha=.1)

    eeg_final = total.copy()#[idx[0]:]
    plt.title("Trial Average From Movement (backwards)", fontsize=20)

# **************************************
# Plot single neuron signals 
# **************************************
if True:
    clrs = ['blue','red','green','magenta','brown']

    trial_selected = 0

    neuron_ave = []
    # loop over each decision and aveage etc.
    for n in range(n_decisions):
        arrays = np.zeros((n_trials,n_time_steps),'float32')+np.nan

        # find when decision was made:
        idx = np.where(np.isnan(evidence[trial_selected,n])==False)[0]
        ts = np.where(decision_reached)[0]
        for t in ts:
            temp=evidence[t,n,:idx[-1]].copy()
            temp+= urgency[:idx[-1]]
            
            x = np.arange(n_time_steps-temp.shape[0],n_time_steps,1)
            plt.plot(x,temp,alpha=0.2,c=colors[n%len(colors)])
            arrays[t,x]=temp

        total = np.nanmean(arrays,0)
        idx = np.where(np.isnan(total)==False)[0]
        #standard_deviation = np.nanstd(arrays[idx[0]:],0)
        standard_deviation = np.nanstd(arrays,0)[idx[0]:]
        x=np.arange(idx[0],n_time_steps,1)

        plt.plot(x,total[idx[0]:],linewidth=5,c=colors[n%len(colors)], label='single neuron or population activity for choice '+str(n))
        plt.fill_between(x, total[idx[0]:] - standard_deviation, 
                            total[idx[0]:] + standard_deviation,
                            color=colors[n%len(colors)],alpha=.3)
    
        neuron_ave.append(total[idx[0]:])
        
    # EEG THRESHOLDS
    idx3 = np.where(np.isnan(eeg_final)==False)[0]
    means = np.nanmean(eeg_final[idx3][:50])
    print ("Means:", means)
    plt.plot([x[0],x[-1]],[means,means],'r--',linewidth=4, c='black',label='start of trial neural activity')
    plt.plot([x[0],x[-1]],[eeg_final[-1],eeg_final[-1]],':',linewidth=4, c='black')
    
    # plot lines for start and end
    plt.plot([x[0],x[-1]],[0,0],'r--',linewidth=4, c='black')
    plt.plot([x[0],x[-1]],[neuron_ave[0][-1],neuron_ave[0][-1]],':',linewidth=4, c='black')#colors[0])
    plt.plot([x[0],x[-1]],[neuron_ave[1][-1],neuron_ave[1][-1]],':',linewidth=4, c='black', label='end of trial neural activity')#colors[1])
    #plt.plot([x[0],x[-1]],[action_threshold,action_threshold],'r--',linewidth=4, c='black')
        
    plt.xlim([x[0],x[-1]])
    
    plt.legend(fontsize=20)
    plt.xlabel("Time",fontsize=35)
    plt.xticks([])
    plt.yticks([])
    #plt.ylabel("Neural Signal", fontsize=40)
    plt.ylabel("Neuronal Activity\nSingle Neuron / Populations      EEG",fontsize=35)

if True:

    from sklearn.decomposition import PCA
    
    idx = np.where(np.isnan(evidence))
    x=evidence.copy()
    print ("Data x: ", x.shape)

    pca_data = x.transpose(0,2,1).copy()
    print ("pca_data: ", pca_data.shape)
        
    pca_data = pca_data.reshape(-1,pca_data.shape[2])
    print ("pca_data: ", pca_data.shape)

    pca_list = []
    for k in range(pca_data.shape[0]):
        if np.any(np.isnan(pca_data[k]))==False:
            pca_list.append(pca_data[k])
    #
    pca_data=np.array(pca_list)
    print ("Final in pcadata; ", pca_data.shape)

    # run PCA
    pca_data+=np.random.rand(pca_data.shape[0],pca_data.shape[1])*10
    pca = PCA(n_components=2)
    pca.fit(pca_data)
    
    # plot urgency signal prsent
    fig=plt.figure()
    ax2=plt.subplot(1,1,1)    
    for k in range(n_trials):
        temp = evidence[k].copy()
        
        # find nan locations and remove them
        idx = np.where(np.isnan(np.sum(temp,0))==True)[0]
        temp = temp[:,:idx[0]-1]
        temp+= urgency[:idx[0]-1]
        temp = temp.T

        # denoise data
        pcs = pca.transform(temp)
                
        c = np.arange(1,pcs.shape[0]+1)

        no_points = pcs.shape[0]
        if True:
            colors2 = plt.cm.viridis(np.linspace(0,1,no_points))
        
        else:
            if decision_type[k]==0:
                colors2 = plt.cm.Blues(np.linspace(0,1,no_points))
            else:
                colors2 = plt.cm.Reds(np.linspace(0,1,no_points))
        
        ax2.set_prop_cycle('color', colors2)

        for i in range(no_points-1):
            plt.plot(pcs[i:i+2,0], pcs[i:i+2,1])
    
    def manual_colorbar(fig, norm, cmap, **kwargs):
        sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        sm.set_array([])
        return fig.colorbar(sm, **kwargs)

    cmap = matplotlib.cm.viridis
    norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
    manual_colorbar(fig,norm,'viridis')

#     img = np.zeros((10,10))
#     plt.imshow(img)
#     plt.colorbar()

    plt.xlabel("PC1",fontsize=35)
    plt.ylabel("PC2",fontsize=35)
    plt.title("Neuronal Population Code",fontsize=35)
    
    # PLOT EEG BASED VECTORS
    
    fig=plt.figure()
    ax3=plt.subplot(111)
    eeg_arrays2 = eeg_arrays.copy() + np.random.rand(eeg_arrays.shape[0],eeg_arrays.shape[1])*1000-500
    
    for t in range(eeg_arrays.shape[0]):                    
        
        idx = np.where(np.isnan(eeg_arrays[t])==True)[0]
        print (idx)
        temp = eeg_arrays[t][idx[-1]:].copy()
        temp2 = eeg_arrays2[t][idx[-1]:].copy()
        print ("TEMP out: ", temp.shape)
        #c = np.arange(1,temp.shape[0]+1)

        #plt.plot(eeg_arrays[t])
        no_points = temp.shape[0]
        print ("no points: ", no_points)
        colors2 = plt.cm.viridis(np.linspace(0,1,no_points))
        
        ax3.set_prop_cycle('color', colors2)

        for i in range(no_points-1):
            print (t,i)
            print (eeg_arrays[t,i:i+1], eeg_arrays2[t,i:i+1])
            plt.plot([temp[i],temp[i+1]], [temp2[i],temp2[i+1]])
    
plt.show()

<ipython-input-52-9c7dffefad1c>:32: RuntimeWarning: Mean of empty slice
  total = np.nanmean(eeg_arrays,0)+eeg_offset1
/home/cat/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-52-9c7dffefad1c>:70: RuntimeWarning: Mean of empty slice
  total = np.nanmean(arrays,0)
/home/cat/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-52-9c7dffefad1c>:70: RuntimeWarning: Mean of empty slice
  total = np.nanmean(arrays,0)
/home/cat/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-52-9c7dffefad1c>:70: RuntimeWarning: Mean of empty slice
  total = np.nanmean(arrays,0)
/h

Means: 933.5525
Data x:  (25, 10, 3000)
pca_data:  (25, 3000, 10)
pca_data:  (75000, 10)
Final in pcadata;  (4166, 10)
[   0    1    2 ... 2807 2808 2809]
TEMP out:  (191,)
no points:  191
0 0
[nan] [nan]
0 1
[nan] [nan]
0 2
[nan] [nan]
0 3
[nan] [nan]
0 4
[nan] [nan]
0 5
[nan] [nan]
0 6
[nan] [nan]
0 7
[nan] [nan]
0 8
[nan] [nan]
0 9
[nan] [nan]
0 10
[nan] [nan]
0 11
[nan] [nan]
0 12
[nan] [nan]
0 13
[nan] [nan]
0 14
[nan] [nan]
0 15
[nan] [nan]
0 16
[nan] [nan]
0 17
[nan] [nan]
0 18
[nan] [nan]
0 19
[nan] [nan]
0 20
[nan] [nan]
0 21
[nan] [nan]
0 22
[nan] [nan]
0 23
[nan] [nan]
0 24
[nan] [nan]
0 25
[nan] [nan]
0 26
[nan] [nan]
0 27
[nan] [nan]
0 28
[nan] [nan]
0 29
[nan] [nan]
0 30
[nan] [nan]
0 31
[nan] [nan]
0 32
[nan] [nan]
0 33
[nan] [nan]
0 34
[nan] [nan]
0 35
[nan] [nan]
0 36
[nan] [nan]
0 37
[nan] [nan]
0 38
[nan] [nan]
0 39
[nan] [nan]
0 40
[nan] [nan]
0 41
[nan] [nan]
0 42
[nan] [nan]
0 43
[nan] [nan]
0 44
[nan] [nan]
0 45
[nan] [nan]
0 46
[nan] [nan]
0 47
[nan] [nan]
0 48


3 19
[nan] [nan]
3 20
[nan] [nan]
3 21
[nan] [nan]
3 22
[nan] [nan]
3 23
[nan] [nan]
3 24
[nan] [nan]
3 25
[nan] [nan]
3 26
[nan] [nan]
3 27
[nan] [nan]
3 28
[nan] [nan]
3 29
[nan] [nan]
3 30
[nan] [nan]
3 31
[nan] [nan]
3 32
[nan] [nan]
3 33
[nan] [nan]
3 34
[nan] [nan]
3 35
[nan] [nan]
3 36
[nan] [nan]
3 37
[nan] [nan]
3 38
[nan] [nan]
3 39
[nan] [nan]
3 40
[nan] [nan]
3 41
[nan] [nan]
3 42
[nan] [nan]
3 43
[nan] [nan]
3 44
[nan] [nan]
3 45
[nan] [nan]
3 46
[nan] [nan]
3 47
[nan] [nan]
3 48
[nan] [nan]
3 49
[nan] [nan]
3 50
[nan] [nan]
3 51
[nan] [nan]
3 52
[nan] [nan]
3 53
[nan] [nan]
3 54
[nan] [nan]
3 55
[nan] [nan]
3 56
[nan] [nan]
3 57
[nan] [nan]
3 58
[nan] [nan]
3 59
[nan] [nan]
3 60
[nan] [nan]
3 61
[nan] [nan]
3 62
[nan] [nan]
3 63
[nan] [nan]
3 64
[nan] [nan]
3 65
[nan] [nan]
3 66
[nan] [nan]
3 67
[nan] [nan]
3 68
[nan] [nan]
3 69
[nan] [nan]
3 70
[nan] [nan]
3 71
[nan] [nan]
3 72
[nan] [nan]
3 73
[nan] [nan]
3 74
[nan] [nan]
3 75
[nan] [nan]
3 76
[nan] [nan]
3 77
[nan] [na

7 60
[nan] [nan]
7 61
[nan] [nan]
7 62
[nan] [nan]
7 63
[nan] [nan]
7 64
[nan] [nan]
7 65
[nan] [nan]
7 66
[nan] [nan]
7 67
[nan] [nan]
7 68
[nan] [nan]
7 69
[nan] [nan]
7 70
[nan] [nan]
7 71
[nan] [nan]
7 72
[nan] [nan]
7 73
[nan] [nan]
7 74
[nan] [nan]
7 75
[nan] [nan]
7 76
[nan] [nan]
7 77
[nan] [nan]
7 78
[nan] [nan]
7 79
[nan] [nan]
7 80
[nan] [nan]
7 81
[nan] [nan]
7 82
[nan] [nan]
7 83
[nan] [nan]
7 84
[nan] [nan]
7 85
[nan] [nan]
7 86
[nan] [nan]
7 87
[nan] [nan]
7 88
[nan] [nan]
7 89
[nan] [nan]
7 90
[nan] [nan]
7 91
[nan] [nan]
7 92
[nan] [nan]
7 93
[nan] [nan]
7 94
[nan] [nan]
7 95
[nan] [nan]
7 96
[nan] [nan]
7 97
[nan] [nan]
7 98
[nan] [nan]
7 99
[nan] [nan]
7 100
[nan] [nan]
7 101
[nan] [nan]
7 102
[nan] [nan]
7 103
[nan] [nan]
7 104
[nan] [nan]
7 105
[nan] [nan]
7 106
[nan] [nan]
7 107
[nan] [nan]
7 108
[nan] [nan]
7 109
[nan] [nan]
7 110
[nan] [nan]
7 111
[nan] [nan]
7 112
[nan] [nan]
7 113
[nan] [nan]
7 114
[nan] [nan]
7 115
[nan] [nan]
7 116
[nan] [nan]
7 117
[nan] [n

11 121
[nan] [nan]
11 122
[nan] [nan]
11 123
[nan] [nan]
11 124
[nan] [nan]
11 125
[nan] [nan]
11 126
[nan] [nan]
11 127
[nan] [nan]
11 128
[nan] [nan]
11 129
[nan] [nan]
11 130
[nan] [nan]
11 131
[nan] [nan]
11 132
[nan] [nan]
11 133
[nan] [nan]
11 134
[nan] [nan]
11 135
[nan] [nan]
11 136
[nan] [nan]
11 137
[nan] [nan]
11 138
[nan] [nan]
11 139
[nan] [nan]
11 140
[nan] [nan]
11 141
[nan] [nan]
11 142
[nan] [nan]
11 143
[nan] [nan]
11 144
[nan] [nan]
11 145
[nan] [nan]
11 146
[nan] [nan]
11 147
[nan] [nan]
11 148
[nan] [nan]
11 149
[nan] [nan]
11 150
[nan] [nan]
11 151
[nan] [nan]
11 152
[nan] [nan]
[   0    1    2 ... 2843 2844 2845]
TEMP out:  (155,)
no points:  155
12 0
[nan] [nan]
12 1
[nan] [nan]
12 2
[nan] [nan]
12 3
[nan] [nan]
12 4
[nan] [nan]
12 5
[nan] [nan]
12 6
[nan] [nan]
12 7
[nan] [nan]
12 8
[nan] [nan]
12 9
[nan] [nan]
12 10
[nan] [nan]
12 11
[nan] [nan]
12 12
[nan] [nan]
12 13
[nan] [nan]
12 14
[nan] [nan]
12 15
[nan] [nan]
12 16
[nan] [nan]
12 17
[nan] [nan]
12 18
[n

15 23
[nan] [nan]
15 24
[nan] [nan]
15 25
[nan] [nan]
15 26
[nan] [nan]
15 27
[nan] [nan]
15 28
[nan] [nan]
15 29
[nan] [nan]
15 30
[nan] [nan]
15 31
[nan] [nan]
15 32
[nan] [nan]
15 33
[nan] [nan]
15 34
[nan] [nan]
15 35
[nan] [nan]
15 36
[nan] [nan]
15 37
[nan] [nan]
15 38
[nan] [nan]
15 39
[nan] [nan]
15 40
[nan] [nan]
15 41
[nan] [nan]
15 42
[nan] [nan]
15 43
[nan] [nan]
15 44
[nan] [nan]
15 45
[nan] [nan]
15 46
[nan] [nan]
15 47
[nan] [nan]
15 48
[nan] [nan]
15 49
[nan] [nan]
15 50
[nan] [nan]
15 51
[nan] [nan]
15 52
[nan] [nan]
15 53
[nan] [nan]
15 54
[nan] [nan]
15 55
[nan] [nan]
15 56
[nan] [nan]
15 57
[nan] [nan]
15 58
[nan] [nan]
15 59
[nan] [nan]
15 60
[nan] [nan]
15 61
[nan] [nan]
15 62
[nan] [nan]
15 63
[nan] [nan]
15 64
[nan] [nan]
15 65
[nan] [nan]
15 66
[nan] [nan]
15 67
[nan] [nan]
15 68
[nan] [nan]
15 69
[nan] [nan]
15 70
[nan] [nan]
15 71
[nan] [nan]
15 72
[nan] [nan]
15 73
[nan] [nan]
15 74
[nan] [nan]
15 75
[nan] [nan]
15 76
[nan] [nan]
15 77
[nan] [nan]
15 78
[nan

19 70
[nan] [nan]
19 71
[nan] [nan]
19 72
[nan] [nan]
19 73
[nan] [nan]
19 74
[nan] [nan]
19 75
[nan] [nan]
19 76
[nan] [nan]
19 77
[nan] [nan]
19 78
[nan] [nan]
19 79
[nan] [nan]
19 80
[nan] [nan]
19 81
[nan] [nan]
19 82
[nan] [nan]
19 83
[nan] [nan]
19 84
[nan] [nan]
19 85
[nan] [nan]
19 86
[nan] [nan]
19 87
[nan] [nan]
19 88
[nan] [nan]
19 89
[nan] [nan]
19 90
[nan] [nan]
19 91
[nan] [nan]
19 92
[nan] [nan]
19 93
[nan] [nan]
19 94
[nan] [nan]
19 95
[nan] [nan]
19 96
[nan] [nan]
19 97
[nan] [nan]
19 98
[nan] [nan]
19 99
[nan] [nan]
19 100
[nan] [nan]
19 101
[nan] [nan]
19 102
[nan] [nan]
19 103
[nan] [nan]
19 104
[nan] [nan]
19 105
[nan] [nan]
19 106
[nan] [nan]
19 107
[nan] [nan]
19 108
[nan] [nan]
19 109
[nan] [nan]
19 110
[nan] [nan]
19 111
[nan] [nan]
19 112
[nan] [nan]
19 113
[nan] [nan]
19 114
[nan] [nan]
19 115
[nan] [nan]
19 116
[nan] [nan]
19 117
[nan] [nan]
19 118
[nan] [nan]
19 119
[nan] [nan]
19 120
[nan] [nan]
19 121
[nan] [nan]
19 122
[nan] [nan]
19 123
[nan] [nan]
19 1

22 136
[nan] [nan]
22 137
[nan] [nan]
22 138
[nan] [nan]
22 139
[nan] [nan]
22 140
[nan] [nan]
22 141
[nan] [nan]
22 142
[nan] [nan]
22 143
[nan] [nan]
[   0    1    2 ... 2844 2845 2846]
TEMP out:  (154,)
no points:  154
23 0
[nan] [nan]
23 1
[nan] [nan]
23 2
[nan] [nan]
23 3
[nan] [nan]
23 4
[nan] [nan]
23 5
[nan] [nan]
23 6
[nan] [nan]
23 7
[nan] [nan]
23 8
[nan] [nan]
23 9
[nan] [nan]
23 10
[nan] [nan]
23 11
[nan] [nan]
23 12
[nan] [nan]
23 13
[nan] [nan]
23 14
[nan] [nan]
23 15
[nan] [nan]
23 16
[nan] [nan]
23 17
[nan] [nan]
23 18
[nan] [nan]
23 19
[nan] [nan]
23 20
[nan] [nan]
23 21
[nan] [nan]
23 22
[nan] [nan]
23 23
[nan] [nan]
23 24
[nan] [nan]
23 25
[nan] [nan]
23 26
[nan] [nan]
23 27
[nan] [nan]
23 28
[nan] [nan]
23 29
[nan] [nan]
23 30
[nan] [nan]
23 31
[nan] [nan]
23 32
[nan] [nan]
23 33
[nan] [nan]
23 34
[nan] [nan]
23 35
[nan] [nan]
23 36
[nan] [nan]
23 37
[nan] [nan]
23 38
[nan] [nan]
23 39
[nan] [nan]
23 40
[nan] [nan]
23 41
[nan] [nan]
23 42
[nan] [nan]
23 43
[nan] [n

In [56]:


# plot single trial example
if True:
    fig=plt.figure()
    ax1=plt.subplot(1,1,1)
    # plot sum + sd of the data
    ctr = np.where(np.isnan(evidence[0,0])==False)[0]
    ctr = ctr[-1]


    # plot single example
    for p in range(n_decisions):
        ax1.plot(evidence[0,p,:ctr]
                +urgency[:ctr],c=colors[p],
                     alpha=1,linewidth=5,  label="Action "+str(p+1))

    # plot stimulus noise
    ax1.plot(stim_noise[0,:ctr]+600,c='green',linewidth=5)

    # plot urgency signal
    ax1.plot(urgency[:ctr]+900,c='magenta',linewidth=5)


    # plot cumulative EEG signal
    print ("Sum: ", np.sum(evidence,0).shape)
    ax1.plot(np.nansum(evidence[0,:,:ctr],0)+urgency[:ctr]
             +stim_noise[0,:ctr]
             +1200,c='black',linewidth=5)
    #ax1.plot([0,ctr*1.1],[action_threshold+900,action_threshold+900],'r--',c='black')

    ctr-=1
    #for p in range(n_decisions):
    #    plt.plot([ctr-1,ctr],[evidence[p,ctr-1],evidence[p,ctr]],c=clrs[p],linewidth=5,

    ax1.plot([0,ctr*1.1],[awareness_threshold,awareness_threshold],'r--',c='black')
    ax1.plot([0,ctr*1.1],[action_threshold,action_threshold],'r--',c='black')

    # visualize underlying waves
    #sin_f = np.sin(np.arange(ctr)/bumps_frequency)*bumps_scaling*50+np.min(evidence)*1.5
    #plt.plot(sin_f,c='black', linewidth=4)


    # plotting data
    x_width = 1.05
    #ax.axhspan(np.nanmin(sin_f)*1.1, awareness, alpha=0.1, color='black')
    ax1.axhspan(awareness_threshold,action_threshold, alpha=0.1, color='red')
    ax1.axhspan(-200,awareness_threshold, alpha=0.1, color='black')

    # formats
    #plt.ylim(np.nanmin(sin_f)*1.1,action_time*x_width*1.2)

    plt.ylabel("Evidence",fontsize=30)
    ax1.set_xticks([])
    #ax.set_yticks([])
    #plt.yticks(fontsize=30)
    plt.legend(fontsize=30)
    plt.xlim(0,ctr*x_width)


Sum:  (10, 3000)


In [847]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll



def colorline(ax,
        x, y, z=None, cmap='copper', norm=plt.Normalize(0.0, 1.0),
        linewidth=3, alpha=1.0):
    """
    http://nbviewer.ipython.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb
    http://matplotlib.org/examples/pylab_examples/multicolored_line.html
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    """

    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))

    # Special case if a single number:
    # to check for numerical input -- this is a hack
    if not hasattr(z, "__iter__"):
        z = np.array([z])

    z = np.asarray(z)

    segments = make_segments(x, y)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap, norm=norm,
                              linewidth=linewidth, alpha=alpha)

    ax = plt.gca()
    ax.add_collection(lc)

    return lc

def make_segments(x, y):
    """
    Create list of line segments from x and y coordinates, in the correct format
    for LineCollection: an array of the form numlines x (points per line) x 2 (x
    and y) array
    """

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    return segments

multicolored_lines()

TypeError: colorline() missing 1 required positional argument: 'y'

In [815]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


def arrowplot(axes, x, y, nArrs=30, mutateSize=10, color='gray', markerStyle='o'): 
    '''arrowplot : plots arrows along a path on a set of axes
        axes   :  the axes the path will be plotted on
        x      :  list of x coordinates of points defining path
        y      :  list of y coordinates of points defining path
        nArrs  :  Number of arrows that will be drawn along the path
        mutateSize :  Size parameter for arrows
        color  :  color of the edge and face of the arrow head
        markerStyle : Symbol
    
        Bugs: If a path is straight vertical, the matplotlab FanceArrowPatch bombs out.
          My kludge is to test for a vertical path, and perturb the second x value
          by 0.1 pixel. The original x & y arrays are not changed
    
        MHuster 2016, based on code by 
    '''
    # recast the data into numpy arrays
    x = np.array(x, dtype='f')
    y = np.array(y, dtype='f')
    nPts = len(x)

    # Plot the points first to set up the display coordinates
    axes.plot(x,y, markerStyle, ms=5, color=color)

    # get inverse coord transform
    inv = axes.transData.inverted()

    # transform x & y into display coordinates
    # Variable with a 'D' at the end are in display coordinates
    xyDisp = np.array(axes.transData.transform(list(zip(x,y))))
    xD = xyDisp[:,0]
    yD = xyDisp[:,1]

    # drD is the distance spanned between pairs of points
    # in display coordinates
    dxD = xD[1:] - xD[:-1]
    dyD = yD[1:] - yD[:-1]
    drD = np.sqrt(dxD**2 + dyD**2)

    # Compensating for matplotlib bug
    dxD[np.where(dxD==0.0)] = 0.1


    # rtotS is the total path length
    rtotD = np.sum(drD)

    # based on nArrs, set the nominal arrow spacing
    arrSpaceD = rtotD / nArrs

    # Loop over the path segments
    iSeg = 0
    while iSeg < nPts - 1:
        # Figure out how many arrows in this segment.
        # Plot at least one.
        nArrSeg = max(1, int(drD[iSeg] / arrSpaceD + 0.5))
        xArr = (dxD[iSeg]) / nArrSeg # x size of each arrow
        segSlope = dyD[iSeg] / dxD[iSeg]
        # Get display coordinates of first arrow in segment
        xBeg = xD[iSeg]
        xEnd = xBeg + xArr
        yBeg = yD[iSeg]
        yEnd = yBeg + segSlope * xArr
        # Now loop over the arrows in this segment
        for iArr in range(nArrSeg):
            # Transform the oints back to data coordinates
            xyData = inv.transform(((xBeg, yBeg),(xEnd,yEnd)))
            # Use a patch to draw the arrow
            # I draw the arrows with an alpha of 0.5
            p = patches.FancyArrowPatch( 
                xyData[0], xyData[1], 
                arrowstyle='simple',
                mutation_scale=mutateSize,
                color=color, alpha=0.5)
            axes.add_patch(p)
            # Increment to the next arrow
            xBeg = xEnd
            xEnd += xArr
            yBeg = yEnd
            yEnd += segSlope * xArr
        # Increment segment number
        iSeg += 1
    


In [3]:
###############################################################
################ LOAD SINGLE TRIAL DATA #######################
###############################################################

fname = '/media/cat/4TBSSD/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_singletrials.npy'
#fname = '/media/cat/4TBSSD/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_3.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_singletrials.npy'
#data_singles = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_04code_singletrials.npy')
data_singles = np.load(fname)
print ("loaded data for single trials: ", data_singles.shape)

root_dir = os.path.split(fname)[0]+'/'
try:
    os.mkdir(root_dir+'/analysis/')
except:
    pass


loaded data for single trials:  (63, 601, 128, 128)


In [4]:
###############################################################
################ LOAD RANDOM TRIAL DATA #######################
###############################################################
#data_random = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_10.0sec_butterworth_globalAverage_0.1hz_6.0hz_randomcode_singletrials.npy')
data_random = np.load(fname.replace('04code','randomcode'))

print ("Random data loaded: ", data_random.shape)


Random data loaded:  (51, 601, 128, 128)


### SINGLE AREA ANALYSES

In [28]:
###############################################################
################ MASK DATA TO A SINGLE AREA ###################
###############################################################
area_masks = np.load('/home/cat/areas.npy',allow_pickle=True)
print ("# of areas saved: ", area_masks.shape)
#print (" areas 0: ", area_masks[0])
#names = [['"Primary', 'motor', 'area,', 'Layer', '2/3"'], ['"Secondary', 'motor', 'area,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'lateral', 'agranular', 'part,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'dorsal', 'part,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'ventral', 'part,', 'layer', '2/3"']]
#[['"Primary', 'motor', 'area,', 'Layer', '2/3"'], ['"Secondary', 'motor', 'area,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'dorsal', 'part,', 'layer', '2/3"'], ['"Retrosplenial', 'area,', 'ventral', 'part,', 'layer', '2/3"']]
names = np.load('/home/cat/area_names.npy',allow_pickle=True)
print ("all areas: ", names)
print (data_singles.shape)

selected_area = [0,1,2,3,5,6,7]
print ("area selected: ", names[selected_area])

prefix = ''
try:
    for k in range(len(names[selected_area])):
        #print (names[selected_area][k].replace('"',''))
        prefix=prefix+"_"+names[selected_area][k].replace('"','')
    prefix = (prefix[1:]).replace(",","").replace('"',"").replace("/","")+"_"
except:
    prefix = 'all_core_'
print (prefix)

# of areas saved:  (10, 128, 128)
all areas:  [list(['"Primary', 'motor', 'area,', 'Layer', '2/3"'])
 list(['"Secondary', 'motor', 'area,', 'layer', '2/3"'])
 list(['"Primary', 'somatosensory', 'area,', 'lower', 'limb,', 'layer', '2/3"'])
 list(['"Primary', 'somatosensory', 'area,', 'upper', 'limb,', 'layer', '2/3"'])
 list(['"Primary', 'visual', 'area,', 'layer', '2/3"'])
 list(['"Retrosplenial', 'area,', 'lateral', 'agranular', 'part,', 'layer', '2/3"'])
 list(['"Retrosplenial', 'area,', 'dorsal', 'part,', 'layer', '2/3"'])
 list(['"Retrosplenial', 'area,', 'ventral', 'part,', 'layer', '2/3"'])
 list(['"Anterior', 'area,', 'layer', '2/3"'])
 list(['"Rostrolateral', 'area,', 'layer', '2/3"'])]
(63, 601, 128, 128)
area selected:  [list(['"Primary', 'motor', 'area,', 'Layer', '2/3"'])
 list(['"Secondary', 'motor', 'area,', 'layer', '2/3"'])
 list(['"Primary', 'somatosensory', 'area,', 'lower', 'limb,', 'layer', '2/3"'])
 list(['"Primary', 'somatosensory', 'area,', 'upper', 'limb,', 'lay

In [6]:
###############################################################
################ MASK DATA TO A SINGLE AREA ###################
###############################################################
# maske real data

if False: # single areas:
    area_mask_selected = area_masks[selected_area]+1
    print ("area mask selected: ", area_mask_selected)
    print ("area mask selected: ", area_mask_selected.shape)
if False:
    #area_mask_selected = np.zeros((128,128),'float32')
    #print ("Area mask: ", area_mask_selected.shape)
    print ("selected areas: ", selected_area)
    area_masks_temp = area_masks
    area_masks_temp+=1
    #print (np.dstack(area_masks[selected_area]).shape)
    area_mask_selected = area_masks_temp[selected_area[0]]
    for s in selected_area[1:]:
        print (s)
        area_mask_selected = np.nansum(np.dstack((area_mask_selected, area_masks_temp[s])),2)
    
    #plt.imshow(area_masks[0])
    #area_mask_selected = np.nansum(np.dstack((area_masks[4],area_masks[5],area_masks[6])),2)
    #area_mask_selected = np.nansum(np.dstack((area_masks[2],area_masks[3])),2)
    print (area_mask_selected.shape)

    plt.imshow(area_mask_selected)
    

# mask areas not involved
data_singles_masked = data_singles.copy()
data_random_masked = data_random.copy()
for k in range(data_singles.shape[0]):
    if k %10==0:
        print ("trial", k)
    for p in range(data_singles.shape[1]):
        data_singles_masked[k,p]=data_singles[k,p]*area_mask_selected
    #print (data_singles[k,p][mask_single_area].shape)
        
for k in range(data_random.shape[0]):
    if k %10==0:
        print ("trial", k)
    for p in range(data_random.shape[1]):
        data_random_masked[k,p]=data_random[k,p]*area_mask_selected

selected areas:  [0, 1, 2, 3, 5, 6, 7]
1
2
3
5
6
7
(128, 128)
trial 0
trial 10
trial 20
trial 30
trial 40
trial 50
trial 60
trial 0
trial 10
trial 20
trial 30
trial 40
trial 50


In [7]:
###############################################################
################ SUM PIXELS IN SINGLE AREA ####################
###############################################################
def sum_pixels_in_single_area(data):

    data_out = data.copy()
    print (" # of trials: ", data.shape[0])
    
    #print ("mask_single_area: ", mask_single_area.shape, mask_single_area)
    
    trial_courses = []
    for k in range(data.shape[0]):
        if k%10==0:
            print ("computing trial: ", k)
        time_courses_local = []

        # convert to 1D vector to mask faster
        trial_courses.append([])
        for p in range(data.shape[1]):
            trial_courses[k].append(
                            data[k,p].sum()
                            )
            print (data[k,p])
            print ("res: ", data[k,p][mask_single_area].shape)
            print ("data out: ", data.shape, data[k,p].shape)
            print (k,p)
            break
#         idx = np.where()[0]

#             # only keep areas that have at least 10 pixels
#             if idx.shape[0]>10:
#                 #print ("Area: ", id_)
#                 area_ids.append(id_)#print ("Areas: ", id_)
#                 #print (data1D[:,idx].shape)

#                 temp = data1D[:,idx]

#                 if False:
#                     # compute DFF
#                     F0 = np.nanmean(temp,axis=0)
#                     dFF = (data1D[:,idx]-F0)/F0
#                 else:
#                     # skip dFF computation;
#                     dFF = temp
                
#                 # save average of all pixesl post DFF
#                 time_courses_local.append(np.nanmean(dFF, axis=1))

#             #all_times.append(time_courses_local)
#         trial_courses.append(time_courses_local)

    trial_courses = np.float32(trial_courses)
    print ("# trials, # areas, # times: ", trial_courses.shape)
    print ("area ids: ", area_ids.shape)

    return area_ids, trial_courses

print ("starting data shape [# trials, # time points, width, height]: ", data_singles_masked.shape)
# accumulate mean activity in each ROI
# input data shape: [# trials, # times, width, height]
trial_courses = np.nansum(np.nansum(data_singles_masked,3),2)[:, None]
print ("trial coruses single area: ", trial_courses.shape)

#trial_courses = sum_pixels_in_single_area(data_singles)
trial_courses_random = np.nansum(np.nansum(data_random_masked,3),2)[:, None] #data_random.sum(3).sum(2)[:,None]
#trial_courses_random = sum_pixels_in_single_area(data_random)
#print (" randomized trial time courses: ", trial_courses_random.shape)

print ("trial courses singel area random: ", trial_courses_random.shape)


starting data shape [# trials, # time points, width, height]:  (63, 601, 128, 128)
trial coruses single area:  (63, 1, 601)
trial courses singel area random:  (51, 1, 601)


In [8]:
# #print (trial_courses[0])
# ax=plt.subplot(2,2,1)
# temp = data_singles[:10,:,45,100].T
# print (temp.shape)
# print (temp)
# #plt.plot(data_singles[:10,:,64,64].T)
# plt.imshow(data_singles[0,300])
# ax=plt.subplot(2,2,2)
# plt.plot(trial_courses_random[:10].squeeze().T)

# ax=plt.subplot(2,2,3)
# plt.imshow(data_random_masked[0,300])

# plt.show()

### MULTI AREA ANALYSIS

In [9]:
###############################################################
################ SUM PIXELS IN MULTIPOLE AREAS AREA ####################
###############################################################
if False:
    prefix="all_areas_"
    # accumulate mean activity in each ROI
    # input data shape: [# trials, # times, width, height]
    area_ids, trial_courses = sum_pixels_in_registered_mask(data_singles, maskwarp)

    # compute random trial time courses
    _, trial_courses_random = sum_pixels_in_registered_mask(data_random, maskwarp)
    print (" randomized trial time courses: ", trial_courses_random.shape)
    print (trial_courses_random.shape)

### REMOVE INFINITES FROM MULTI- or SINGLE AREA DATA

In [10]:
#####################################################################
######## REMOVE INFINITIES, NANS ETC FROM DATA ######################
#####################################################################
trial_courses_fixed, trial_courses_random_fixed = fix_trials(trial_courses, trial_courses_random)
print ("trial_courses_fixed: ", trial_courses_fixed.shape)
print ('trial_courses_random_fixed: ', trial_courses_random_fixed.shape)
print("DONE")
#plt.show()


trial_courses_fixed:  (63, 1, 601)
trial_courses_random_fixed:  (51, 1, 601)
DONE


In [11]:
# #print (trial_courses[0])
# ax=plt.subplot(1,2,1)
# plt.plot(trial_courses[:10].squeeze().T)
# ax=plt.subplot(1,2,2)
# plt.plot(trial_courses_random_fixed[:10].squeeze().T)
# plt.show()

In [12]:
# generate 80% - 20% training - testing datasets
def generate_training_data(trial_courses_fixed, trial_courses_random_fixed):
    selected_trials = np.arange(int(trial_courses_fixed.shape[0]*0.8)) # of trials to separate the train vs test data
    selected_trials_random = np.arange(int(trial_courses_random_fixed.shape[0]*0.8)) # of trials to separate the train vs test data

    trial_courses_fixed_ids = []
    trial_courses_random_fixed_ids = []
    ctr=0
    while True:
        trial_courses_fixed_ids.append(selected_trials+ctr)
        trial_courses_random_fixed_ids.append(selected_trials_random+ctr)

        ctr+=1
        if (ctr+selected_trials.shape[0]>=trial_courses_fixed.shape[0]):
            break
        if (ctr+selected_trials_random.shape[0]>=trial_courses_random_fixed.shape[0]):
            break

    return trial_courses_fixed_ids, trial_courses_random_fixed_ids

def normalize_data(data1, data2, random_flag):
    
    #print ("NIORMALIZEION: ", data1.shape, data2.shape)
    data_in = np.vstack((data1,data2))    
    #print ("Data in: ", data_in.shape)
    
    if random_flag:
        idx_random = np.random.choice(np.arange(data_in.shape[0]), size=data_in.shape[0], replace=False)
        #print (idx_random.shape)
        data_in = data_in[idx_random]
        #print ("Data in: ", data_in.shape)

    
    #data_in shaep: (63, 35, 181)
    for k in range(data_in.shape[1]):
        for p in range(data_in.shape[2]):
            temp = data_in[:,k,p]
            #print ("temp re: ", temp)
            temp = (temp-temp.mean(0))/(temp.std(0)+0.00000001) # to avoid nans
            #print ("temp post: ", temp)
            data_in[:,k,p] = temp
    
    data1 = data_in[:data1.shape[0]]
    data2 = data_in[data1.shape[0]:]   
    #print ("POST NIORMALIZEION: ", data1.shape, data2.shape)

    return data1, data2


def make_training_sets_multiple_tests_window2(time, 
                                             time_window,
                                             trial_courses_fixed, 
                                             trial_courses_fixed_ids,
                                             trial_courses_random_fixed, 
                                             trial_courses_random_ids):
    
    # combine good trials with random trials for training sets:
    #time = 0
    #print ("trial_courses_fixed: ", trial_courses_fixed.shape)
    good_trials = trial_courses_fixed[trial_courses_fixed_ids, :,time:time+time_window].reshape(trial_courses_fixed_ids.shape[0], -1)
    #print ("good trials: ", good_trials.shape)
    temp = np.arange(trial_courses_fixed.shape[0])
    idx = np.delete(temp,trial_courses_fixed_ids) # remove the training set trials for testing stage
    test_trials = trial_courses_fixed[idx, :,time:time+time_window].reshape(idx.shape[0], -1)  # select left over trials for testing;
    #print ("good test_trials: ", test_trials.shape)
    
    random_trials = trial_courses_random_fixed[trial_courses_random_ids, :,time:time+time_window].reshape(trial_courses_random_ids.shape[0], -1)
    #print ("random_trials: ", random_trials.shape)
    temp = np.arange(trial_courses_random_fixed.shape[0])
    idx = np.delete(temp,trial_courses_random_ids) # remove training set trials for testing
    test_trials_random = trial_courses_random_fixed[idx, :,time:time+time_window].reshape(idx.shape[0], -1)
    #print ("test_trials_random: ", test_trials_random.shape)
    #print ("test_trials_random: ", test_trials_random.shape)

    # rebalance the data to ensure SVM doesn't overfit; 
    # Cat TODO: this is a big issue in cases where insufficient postiive of negative trials are presnt:
    max_n_trials = min(good_trials.shape[0], random_trials.shape[0])
    good_trials = good_trials[:max_n_trials]
    random_trials = random_trials[:max_n_trials]

    # 
    max_n_trials = min(test_trials.shape[0], test_trials_random.shape[0])
    test_trials = test_trials[:max_n_trials]
    test_trials_random = test_trials_random[:max_n_trials]
    
    
    # make labels
    #y = np.zeros(good_trials.shape[0]+random_trials.shape[0],'int32')
    #y[:good_trials.shape[0]]=1
    y = np.zeros(good_trials.shape[0]+random_trials.shape[0],'int32')
    y[:good_trials.shape[0]]=1

    # concatenate
    X = np.vstack((good_trials,random_trials))

    #rint ("done time: ", time)
    
    
    return X, y, test_trials, test_trials_random



def parallel_svm_multiple_tests2(time, 
                                time_window,
                                trial_courses_fixed, 
                                trial_courses_fixed_ids,
                                trial_courses_random_fixed, 
                                trial_courses_random_ids,
                                random_flag,
                                root_dir):


    # 
    if time%100==0:
        print ("processing time: ", time)
        
    #print ("normalizing data... (TO CHECK THIS STEP)")
    trial_courses_fixed, trial_courses_random_fixed = \
                normalize_data(trial_courses_fixed, trial_courses_random_fixed, random_flag)
    
    #print ("trial_courses_fixed_ids: ", trial_courses_fixed_ids)
    res1 = []
    res2 = []
    sens = []
    spec = []
    accuracy = []
    for k in range(len(trial_courses_fixed_ids)):
        X, y, test_trials, test_trials_random = make_training_sets_multiple_tests_window2(time, 
                                                                                         time_window,
                                                                                         trial_courses_fixed, 
                                                                                         trial_courses_fixed_ids[k],
                                                                                         trial_courses_random_fixed, 
                                                                                         trial_courses_random_ids[k])
        #print ("test trials: ", test_trials.shape)
        #print ("X: ", X.shape)
        #print ("y: ", y.shape)
        #plotting=False
        model = SVC(kernel='linear', C=1)
        model.fit(X, y)
        #plot_decision_regions(X, y, clf=model, legend=2)

        support_vectors = model.support_vectors_        
        #print ("support_vectors: ", support_vectors)
        
        #plt.scatter(support_vectors, support_vectors*0 + 2, color='green')
        
        #print ("X: ", X)
        #print ("y: ", y)
        #plt.scatter(X, X*0-0.5, c=y)
        # test real data
        #print ("test trials : ", test_trials)
        #plt.scatter(test_trials, test_trials*0+0.25,c='blue',label='test trials')

        yfit = model.predict(test_trials)
        true_pos = np.sum(yfit)
        false_neg = (test_trials.shape[0]-true_pos)
        #print ("True pos: ", true_pos, " false neg: ", false_neg)
        res1.append(np.sum(yfit)/float(yfit.shape[0]))

        # test random data
        #print ("test trials random: ", test_trials_random)
        #plt.scatter(test_trials_random, test_trials_random*0+0.5,c='red', label='random_trials')

        yfit = model.predict(test_trials_random)
        false_pos = np.sum(yfit)
        true_neg = (test_trials_random.shape[0]-false_pos)
        #print ("True negative: ", true_neg, " false_pos: ", false_pos)
        res2.append(np.sum(yfit)/float(yfit.shape[0]))
    
        # compute sensitivity:  true positives / (true positives + false negatives)
        sens.append(true_pos / (true_pos+false_neg))
        spec.append(true_neg / (true_neg+false_pos))
        
        all_pos = true_pos + false_neg
        all_neg = false_pos + true_neg
        # compute accuracy
        accuracy_temp = (true_pos+true_neg)/(all_pos+all_neg)
        
        if False:
            print ("")
            print ("SVM run: ", k)
            print ("true_pos :", true_pos)
            print ("true_neg :", true_neg)
            print ("false_neg :", false_neg)
            print ("false_pos :", false_pos)
            print ("all_pos :", all_pos)
            print ("all_neg :", all_neg)
            print ("accuracy :", accuracy_temp)

        # compute accuracy:
        accuracy.append(accuracy_temp)
    
    #return (res1, res2, sens, spec)
    np.save(root_dir + str(time)+'_sens.npy', sens)
    np.save(root_dir + str(time)+'_spec.npy', spec)
    np.save(root_dir + str(time)+'_accuracy.npy', accuracy)
    
    return (res1, res2)
# 
def plot_accuracy2(root_dir, length_rec, 
                   fig, ax,
                   clr, label_in, labels):
    time_window=1
    #fig=plt.figure()
    #ax=plt.subplot(111)
    sample_rate = 30
    window_size = length_rec//sample_rate/2
    spec_array = []
    spec_error = []
    sens_array = []
    sens_error = []
    accuracy_array = []
    accuracy_error = []

    for k in range(length_rec):
        #print (k)
        sens = np.load(root_dir+str(k)+"_sens.npy")
        spec = np.load(root_dir+str(k)+"_spec.npy")
        acc = np.load(root_dir+str(k)+"_accuracy.npy")
        #print (sens, spec)

        # plot real pulls time-series
        sens_array.append(sens.mean(0))
        sens_error.append(sens.std(0))

        # plot randomized prediction time-series
        spec_array.append(spec.mean(0))
        spec_error.append(spec.std(0))

        # plot randomized prediction time-series
        accuracy_array.append(acc.mean(0))
        accuracy_error.append(acc.std(0))

    t = np.arange(len(sens_array))/30.-(length_rec//sample_rate/2)
    # plt.errorbar(t, 
    #              sens_array, sens_error, linestyle='None', marker='^', c='blue', alpha=.7)
    # plt.plot(t, sens_array, c='blue', alpha=.7)

    # plt.errorbar(t, 
    #              spec_array, spec_error, linestyle='None', marker='^', c='red', alpha=.7)
    # plt.plot(t, spec_array, c='red', alpha=.7)

    if False:
        plt.errorbar(t, 
                 accuracy_array, accuracy_error, linestyle='None', marker='^', c=clr, alpha=.7)
    
    from scipy.signal import savgol_filter
    
    yhat = savgol_filter(accuracy_array, 3, 1) #
    plt.plot(t, yhat, c=clr,linewidth=5, alpha=.7)



    #labels = []
    patch_j = mpatches.Patch(color = clr, label = label_in[:-1])
    labels.append(patch_j)
#     patch_j = mpatches.Patch(color = 'red', label = "Motor")
#     labels.append(patch_j)
#     patch_j = mpatches.Patch(color = 'black', label = "All Cortex")
#     labels.append(patch_j)

    plt.ylim(0,1)
    plt.xlim(t[0],t[-1])
    #plt.legend(handles=labels)
    plt.ylim(0,1)
    plt.plot([t[0],t[-1]],[0.5,0.5],'r--',c='black')
    plt.plot([0,0],[0,1],'r--',c='black')
    ax.tick_params(axis = 'both', which = 'major', labelsize = 20)

    plt.suptitle("Sliding time window prediction using "+str(time_window)+" frames = "+str(round(time_window/30.,2))+"sec", fontsize=20)
    plt.xlabel("Time (sec)",fontsize=20)
    plt.suptitle(root_dir, fontsize=20)
    #plt.show()
    return labels, ax

In [13]:
# ################# TRAIN SVM CLASSIFIER OVER THE DATA #################
# make batches of data to train and test over
# ratio of testing 80% train to 20% test is fixed inside function
trial_courses_fixed_ids, trial_courses_random_fixed_ids = \
            generate_training_data(trial_courses_fixed, trial_courses_random_fixed)

# print szies of datasets
#print ("trial courses fixed ids: ", len(trial_courses_fixed_ids))
#print ("trial_courses_random_fixed_ids: ", len(trial_courses_random_fixed_ids))

# start 
random_flag = False  # shuffle data to show baseline
#prefix='secondary_motor'
#if random_flag:
#    prefix = "random_"
    
time_window = 1

from mlxtend.plotting import plot_decision_regions

# Loading some example data
# iris = datasets.load_iris()
# X = iris.data[:, 2]
# X = X[:, None]
# y = iris.target

# # Training a classifier
# svm = SVC(C=0.5, kernel='linear')
# svm.fit(X, y)

# Plotting decision regions


#times = np.arange(0,trial_courses_fixed.shape[2]-time_window)
times = np.arange(0,trial_courses_fixed.shape[2])
if False:
    res = parmap.map(parallel_svm_multiple_tests2, 
                     times, 
                     time_window,
                     trial_courses_fixed, 
                     trial_courses_fixed_ids, # ids of trial sto be used; make sure 
                     trial_courses_random_fixed, 
                     trial_courses_random_fixed_ids,
                     random_flag,
                     root_dir+'/analysis/'+prefix,
                     pm_processes=4)
else:
    res = []
    
    for k in range(len(times)):
    #for k in [0]:
        res.append(parallel_svm_multiple_tests2(times[k], 
                                     time_window,
                     trial_courses_fixed, 
                     trial_courses_fixed_ids, # ids of trial sto be used; make sure 
                     trial_courses_random_fixed, 
                     trial_courses_random_fixed_ids,
                     random_flag,
                     root_dir+'/analysis/'+prefix))
        #break
    
res = np.vstack(res)
print ("DONE")

processing time:  0
processing time:  100
processing time:  200
processing time:  300
processing time:  400
processing time:  500
processing time:  600
DONE


In [27]:
###############################################################
########## PLOT ACCURACY FROM PREVIOUS SAVED DATA #############
###############################################################
names = np.load('/home/cat/area_names.npy',allow_pickle=True)

clrs = ['black','red','blue','yellow','pink','green']
labels = []
fig=plt.figure()
ax=plt.subplot(111)

ctr=0
# plot all data:
label_in = prefix = 'all_areas_'
#label_in = names[selected_area]
labels, ax = plot_accuracy2(root_dir+'/analysis/'+prefix, trial_courses_fixed.shape[2]-1, 
                            fig, ax, clrs[ctr], label_in, labels)
ctr+=1

# plot all other regions
names_prefix = ['all_limb_','Primary_visual_area_layer_23_']
clrs=['green','blue']
#names_prefix = ['all_motor_','all_rs_','Primary_visual_area_layer_23_']
#names_prefix = ['Primary_visual_area_layer_23_']
#names_prefix = ['Anterior_area_layer_23_']
#names_prefix = ['Rostrolateral_area_layer_23_']

#selected_areas = [0,2]
ctr=0
#clrs = ['red','blue','green']

for prefix in names_prefix:
    #print ("area selected: ", names[selected_area])
    print ('prefix: ', prefix)
#     prefix = ''
#     for k in range(len(names[selected_area])):
#         #print (names[selected_area][k].replace('"',''))
#         prefix=prefix+"_"+names[selected_area][k].replace('"','')
#     prefix = (prefix[1:]).replace(",","").replace('"',"").replace("/","")+"_"
#     print (prefix)

    label_in = prefix
    labels, ax = plot_accuracy2(root_dir+'/analysis/'+prefix, trial_courses_fixed.shape[2]-1, 
                            fig, ax, clrs[ctr], label_in, labels)
    ctr+=1
    
# plot cumulative RS plot
# selected_areas = [4]
# prefix="all_rs_"

# label_in = prefix #names[selected_area]
# labels, ax = plot_accuracy2(root_dir+'/analysis/'+prefix, trial_courses_fixed.shape[2]-1, 
#                             fig, ax, 'green', label_in, labels)
# ctr+=1
    
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
ax.axhspan(0.4, 0.6, alpha=0.1, color='black')
ax.legend(handles=labels, fontsize=20)
#plt.ylim(0.3,1.0)
plt.show()

prefix:  all_limb_
prefix:  Primary_visual_area_layer_23_


In [12]:
# plot classification rates
time_window=1
plot_true_vs_random(res, trial_courses_fixed, time_window)


[[0.53846154 0.46153846 0.46153846 ... 0.38461538 0.30769231 0.38461538]
 [0.53846154 0.46153846 0.46153846 ... 0.30769231 0.30769231 0.38461538]
 [0.53846154 0.46153846 0.61538462 ... 0.30769231 0.23076923 0.30769231]
 ...
 [0.53846154 0.38461538 0.53846154 ... 0.61538462 0.38461538 0.46153846]
 [0.53846154 0.46153846 0.69230769 ... 0.61538462 0.46153846 0.38461538]
 [0.69230769 0.46153846 0.61538462 ... 0.76923077 0.30769231 0.30769231]]


In [17]:
# plot precision and recall
# plot_specificity_sensitivity(res, trial_courses_fixed)


In [13]:
# SVM - all areas at once; + time window
times1 = np.arange(0,151,1)
times2 = np.arange(30,181,1)
times = np.vstack((times1,times2)).T
#print (times)
res = parmap.map(parallel_svm_multi_time, times, trial_courses_fixed, 
                 trial_courses_random_fixed,
                 pm_processes=4)

#print (times)
# parallel_svm_multi_time(times[0], trial_courses_fixed, 
#                         trial_courses_random_fixed)

res = np.vstack(res)

times  [ 0 30]
times  times [10 40] 
[30 60]
times  [40 70]
times  [50 80]
times times   [20 50][60 90]

times  [ 70 100]
times  times  [ 80 110]times  
[ 90 120]
[100 130]
times times  [120 150]
 times [110 140][130 160] times 

 [140 170]
times  [150 180]


NameError: name 'real_data' is not defined

In [22]:
# SVM time window; area specific

times1 = np.arange(0,151,1)
times2 = np.arange(30,181,1)
times = np.vstack((times1,times2)).T

areas = np.arange(35)
#print (times)
res = parmap.map(parallel_svm_multi_time_multi_area, 
                 times, 
                 areas,
                 trial_courses_fixed, 
                 trial_courses_random_fixed,
                 pm_processes=4)

#print (times)
# parallel_svm_multi_time(times[0], trial_courses_fixed, 
#                         trial_courses_random_fixed)

res = np.array(res)



times  [ 0 30]
times  [10 40]
times  [20 50]
times  [30 60]
done times:  [ 0 30]
times  [ 1 31]
done times:  [10 40]
times  [11 41]
done times:  [20 50]
times  [21 51]
done times:  [30 60]
times  [31 61]
done times:  [ 1 31]
times  [ 2 32]
done times:  [11 41]
times  [12 42]
done times:  [21 51]
times  [22 52]
done times:  [31 61]
times  [32 62]
done times:  [ 2 32]
times  [ 3 33]
done times:  [12 42]
times  [13 43]
done times:  [22 52]
times  [23 53]
done times:  [32 62]
times  [33 63]
done times:  [ 3 33]
times  [ 4 34]
done times:  [13 43]
times  [14 44]
done times:  [23 53]
times  [24 54]
done times:  [33 63]
times  [34 64]
done times:  [ 4 34]
times  [ 5 35]
done times:  [14 44]
times  [15 45]
done times:  [24 54]
times  [25 55]
done times:  [34 64]
times  [35 65]
done times:  [ 5 35]
times  [ 6 36]
done times:  [15 45]
times  [16 46]
done times:  [25 55]
times  [26 56]
done times:  [35 65]
times  [36 66]
done times:  [ 6 36]
times  [ 7 37]
done times:  [16 46]
done times:  [26 56

In [21]:
# PLOTS FOR SINGLE AREA / CUMULATIVE AREAS
t = np.arange(res.shape[0])/30-3
plt.plot(t,res[:,0],c='blue')
plt.plot(t,res[:,1],c='red')
plt.title("Decoding rewarded trials (blue); random trials (red)"+
         "\nUsing single 1sec of data (30 frames @30Hz) bins",fontsize=20)
plt.plot([-3,3],[0.5,0.5],'r--',c='black')
plt.plot([0,0],[0,1],'r--',c='black')
plt.xlabel("Time (sec)",fontsize=20)
plt.ylabel("Probability (dash=chance)",fontsize=20)

plt.show()


In [67]:
# PLOTS PER AREA

t = np.arange(res.shape[0])/30-3
for k in range(35):
    ax=plt.subplot(5,7,k+1)
# plt.plot(t,real_data,c='blue')
    # plt.plot(t,random_data,c='red')
    
    plt.plot(t,res[:,0,k],c='blue')
    plt.plot(t,res[:,1,k],c='red')
    plt.plot([-3,3],[0.5,0.5],'r--',c='black')
    plt.plot([0,0],[0,1],'r--',c='black')
    if k<34:
        plt.xticks([])
    plt.title("Area: "+str(k))
    
plt.suptitle("Decoding rewarded trials (blue); random trials (red)"+
             "\nUsing single 1sec of data (30 frames @30Hz) bins",fontsize=20)
#plt.xlabel("Time (sec)",fontsize=20)
#plt.ylabel("Probability (dash=chance)",fontsize=20)

plt.show()

In [28]:
#test_trials_rewarded = trial_courses_fixed[50:, :,time].reshape(50, -1)
#model = grid.best_estimator_
yfit = model.predict(test_trials)
print ("real result: ", np.sum(yfit)/float(yfit.shape[0]))

yfit = model.predict(test_trials_random)
print ("random result: ", np.sum(yfit)/float(yfit.shape[0]))




real result:  0.6923076923076923
random result:  0.34


(63, 720)


In [21]:
abstimes = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abstimes.npy')
abspositions = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abspositions.npy')
abscodes = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abscodes.npy')

print (abstimes.shape, abstimes)
print (abspositions.shape, abspositions)
print (abscodes.shape, abscodes)



(156815,) [0.00000000e+00 0.00000000e+00 8.33333333e-03 ... 1.30421401e+03
 1.30422234e+03 1.30423067e+03]
(156815,) [0. 0. 0. ... 0. 0. 0.]
(156815,) [b'0' b'0' b'0' ... b'0' b'0' b'0']


In [22]:
likelihoods = lever[:,2]
# idx = np.where(np.logical_and(likelihoods>0.8, likelihoods<=1.0))[0]
# likelihoods[...]= 0
# likelihoods[idx]=1
print (likelihoods.shape)
# plt.plot(likelihoods)
# plt.show()

idx = np.where(likelihoods<0.8)[0]
lever[idx,0]=np.median(lever[:,0])
lever[idx,1]=np.median(lever[:,1])

# blue = np.asarray([(0, 0, 1, a) for a in likelihoods])
# print (blue)
# red = np.asarray([(1, 0, 0, a) for a in likelihoods])
# print (red)



NameError: name 'lever' is not defined

In [27]:
fig=plt.figure()

# plot DLC traces
#for k in range(len(traces)):
for k in range(2):
    print (traces[0].shape)
    times = np.arange(traces[k].shape[0])/15.
    plt.plot(times, (traces[k][:,0]-np.median(traces[k][:,0]))*3, color=colors[k+1], label=labels[k]+ " - x loc")
    #plt.plot(times, (traces[k][:,1]-np.median(traces[k][:,1]))*3,'r--', color=colors[k+1], label=labels[k]+ " - y loc")

abstimes_shifted = abstimes + 3.4
plt.plot(abstimes_shifted, abspositions, linewidth=4, c='black', label="Lever angle readout")
#print (lever.shape, lever[:,0].shape)

plt.legend(fontsize=20)
plt.show()

(20028, 3)
(20028, 3)


In [ ]:
abstimes = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abstimes.npy')
abspositions = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abspositions.npy')
abscodes = np.load('/media/cat/1TB/data/yuki/iA2/tif_files/IA2pm_Apr22_Week2_30Hz/IA2pm_Apr22_Week2_30Hz_abscodes.npy')
t=np.arange(-3,3,0.03)
plt.plot([t[0], t[-1]], [0,0],c='black')
plt.plot([t[0], t[-1]], [11,11],'r--', c='black')
plt.plot([t[0], t[-1]], [40,40],'r--',c='black')
plt.plot([t[0], t[-1]], [60,60],c='black')
plt.ylim(0,100)
plt.xlim(t[0],t[-1])
plt.show()


In [16]:
data = np.load('/media/cat/1TB/data/synthetic/ari_neighbourchans/data_raw_30000hz_300sec_35hz_to_55hz_templates.npy')
print (data.shape)

(8, 101)
